In [20]:
#importing pertinent libraries
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests, re
import json

In [21]:
#extracting UFC data

#creating a list of unique fighter URLs available on website
fighter_url_list = []
response = requests.get('http://www.ufcstats.com/statistics/fighters?char=a&page=all')
soup = BeautifulSoup(response.content, 'html.parser')

#targeting URLs that contain fighter data
fighters = soup.findAll('a', attrs={'href': re.compile('http:')})
for fighter in fighters:
    fighter_url_list.append(fighter.get('href'))

#removing duplicate URLs
fighter_url_list = list(set(fighter_url_list))
fighter_url_list

['http://www.ufcstats.com/fighter-details/b0550072e5f0afa7',
 'http://www.ufcstats.com/fighter-details/70fa1c64a2c439ef',
 'http://www.ufcstats.com/fighter-details/1e38bba6738b7b10',
 'http://www.ufcstats.com/fighter-details/49b130de1da2e1bf',
 'http://www.ufcstats.com/fighter-details/770b9d4813c25902',
 'http://www.ufcstats.com/fighter-details/19ba965651486013',
 'http://www.ufcstats.com/fighter-details/ee18ff42063174df',
 'http://www.ufcstats.com/fighter-details/a0e75f4a13eb73f1',
 'http://www.ufcstats.com/fighter-details/c0ed7b208197e8de',
 'http://www.ufcstats.com/fighter-details/578ef12674df1e6a',
 'http://www.ufcstats.com/fighter-details/44aa652b181bcf68',
 'http://www.ufcstats.com/fighter-details/e70de1859b7ee78e',
 'http://www.ufcstats.com/fighter-details/33a331684283900f',
 'http://www.ufcstats.com/fighter-details/196ed28337adc630',
 'http://www.ufcstats.com/fighter-details/e76d9f656ceb82ab',
 'http://www.ufcstats.com/fighter-details/cad24459b28592ca',
 'http://www.ufcstats.co

Looks like there are some URLs we don't need in out list. Let's weed them out.

In [22]:
#removing invalid URLs
for url in fighter_url_list:
    if 'fighter-details' not in url:
        fighter_url_list.remove(url)

In [23]:
fighter_url_list

['http://www.ufcstats.com/fighter-details/b0550072e5f0afa7',
 'http://www.ufcstats.com/fighter-details/70fa1c64a2c439ef',
 'http://www.ufcstats.com/fighter-details/1e38bba6738b7b10',
 'http://www.ufcstats.com/fighter-details/49b130de1da2e1bf',
 'http://www.ufcstats.com/fighter-details/770b9d4813c25902',
 'http://www.ufcstats.com/fighter-details/19ba965651486013',
 'http://www.ufcstats.com/fighter-details/ee18ff42063174df',
 'http://www.ufcstats.com/fighter-details/a0e75f4a13eb73f1',
 'http://www.ufcstats.com/fighter-details/c0ed7b208197e8de',
 'http://www.ufcstats.com/fighter-details/578ef12674df1e6a',
 'http://www.ufcstats.com/fighter-details/44aa652b181bcf68',
 'http://www.ufcstats.com/fighter-details/e70de1859b7ee78e',
 'http://www.ufcstats.com/fighter-details/33a331684283900f',
 'http://www.ufcstats.com/fighter-details/196ed28337adc630',
 'http://www.ufcstats.com/fighter-details/e76d9f656ceb82ab',
 'http://www.ufcstats.com/fighter-details/cad24459b28592ca',
 'http://www.ufcstats.co

In [25]:
#creating data frames and lists  
fighter_basic_stats = pd.DataFrame()

name_list = [] 
height_list = []
record_list = []
weight_list = []
reach_list = []
stance_list = []
DOB_list  = []


#creating data frames and lists 
fighter_career_stats = pd.DataFrame()


SLpM_list = []
StrAcc_list = []
SApM_list = []
StrDef_list = []
TDAvg_list = []
TDAcc_list  = []
TDDef_list  = []
SubAvg_list  = []

In [26]:
for url in fighter_url_list:

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    base_stats = soup.findChildren('li', attrs={'class': re.compile('b-list')})
    career_stats = base_stats[5:]
    fights = soup.findAll('a', attrs={'href': re.compile('fight-details')})

    #Retrieving fighter name/appending fighter_basic_stats
    name = soup.findChildren('span', attrs={'class': re.compile('b-content__title-highlight')})
    name_list.append(name[0].text.strip())
    
    #Retrieving fighter record
    record = soup.findChildren('span', attrs={'class': re.compile('b-content__title-record')})
    record_list.append(record[0].text.strip().split(':')[1].strip())
    
    base_stats_titles = soup.findChildren('li', attrs={'class': re.compile('b-list')})

    base_stats = soup.findChildren('li', attrs={'class': re.compile('b-list')})
    
    #feet
    height = int(base_stats[0].text.strip().split(':')[1].strip().split(' ')[0].replace("'", "").replace("--", "0"))
    
    #inches
    if height == 0:
        pass
    else:
        inches = int(base_stats[0].text.strip().split(':')[1].strip().split(' ')[1].replace("'", "").replace('"', '').replace("--", "0"))

    height_to_inches = (height * 12) 
    height_list.append(height_to_inches + inches)
    
    #Weight
    weight = base_stats[1].text.strip().split(':')[1].strip().replace(' lbs.', '')
    weight_list.append(weight)
    
    #Reach
    reach = base_stats[2].text.strip().split(':')[1].strip().replace('"', '')
    reach_list.append(reach)
    
    #Stance
    stance = base_stats[3].text.strip().split(':')[1].strip()
    stance_list.append(stance)
    
    #DOB
    DOB = base_stats[4].text.strip().split(':')[1].strip()
    DOB_list.append(DOB)
    
     #Significant Strkes Landed per Minute
    SLpM = career_stats[0].text.split(':')[1].replace('%', '').strip()
    SLpM_list.append(SLpM)
    
    #Striking Accuracy %
    StrAcc =  career_stats[1].text.split(':')[1].replace('%', '').strip()
    StrAcc_list.append(StrAcc)
    
    #Striking Accuracy %
    SApM = career_stats[1].text.split(':')[1].strip()
    SApM_list.append(SApM)
    
    #Striking Defence %
    StrDef = career_stats[2].text.split(':')[1].replace('%', '').strip()
    StrDef_list.append(StrDef)
    
    #Average Takedowns Landed per 15 min
    TDAvg = career_stats[5].text.split(':')[1].strip()
    TDAvg_list.append(TDAvg)
    
    #Takedown Accuracy
    TDAcc = career_stats[6].text.split(':')[1].strip().replace('%', '')
    TDAcc_list.append(TDAcc)
    
    #Takedown Defense %
    TDDef = career_stats[7].text.split(':')[1].replace('%', '').strip()
    TDDef_list.append(TDDef)
    
    #Average Submissiongs Attempted per 15 min
    SubAvg =  career_stats[8].text.split(':')[1].strip().replace('%', '')  
    SubAvg_list.append(SubAvg)



In [ ]:
for fight in fights:
    fight_holder.append(fight.get('href'))
fight_list.append(fight_holder) 
fight_history['name'] = name_list
fight_history['fight_history'] = fight_list

In [27]:
#populating basic fighter stats dataframe
fighter_basic_stats['name'] = name_list
fighter_basic_stats['height'] = height_list
fighter_basic_stats['record'] = record_list
fighter_basic_stats['weight'] =  weight_list
fighter_basic_stats['reach'] = reach_list
fighter_basic_stats['stance'] = stance_list
fighter_basic_stats['DOB'] = DOB_list

fighter_basic_stats

,name,height,record,weight,reach,stance,DOB
0,Yoshihiro Akiyama,70,14-6-0 (2 NC),170,73,Orthodox,"Jul 29, 1975"
1,Alen Amedovski,70,8-2-0,185,74,Orthodox,"Apr 06, 1988"
2,Alfonso Alcarez,63,1-3-1,145,--,,"Jun 10, 1970"
3,Sarah Alpar,64,8-4-0,135,--,Southpaw,"Jun 01, 1991"
4,Chalid Arrab,71,7-3-0,205,--,Orthodox,"May 28, 1975"
...,...,...,...,...,...,...,...
158,Royce Alger,70,3-2-0,199,--,Orthodox,--
159,Sam Alvey,74,33-14-0 (1 NC),205,75,Southpaw,"May 06, 1986"
160,John Alessio,70,34-16-0 (1 NC),155,72,Orthodox,"Jul 05, 1979"
161,Mike Altman,70,1-4-0,185,--,Orthodox,--


In [29]:
fighter_career_stats['name'] = name_list
fighter_career_stats['SLpM'] = SLpM_list
fighter_career_stats['StrAcc'] = StrAcc_list
fighter_career_stats['SApM'] = SApM_list
fighter_career_stats['StrDef'] = StrDef_list
fighter_career_stats['TDAvg'] = TDAvg_list
fighter_career_stats['TDAcc'] = TDAcc_list
fighter_career_stats['TDDef'] = TDDef_list
fighter_career_stats['SubAvg'] = SubAvg_list

fighter_career_stats

ValueError: Length of values does not match length of index

In [31]:
len(SLpM_list)

0

In [19]:
fight_list = []
fight_history = pd.DataFrame()

for url in fighter_url_list: 
   
    fight_holder = []
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    fights = soup.findAll('a', attrs={'href': re.compile('fight-details')})
    
    for fight in fights:
        fight_holder.append(fight.get('href'))
    fight_list.append(fight_holder) 
fight_history['name'] = name_list
fight_history['fight_history'] = fight_list

ConnectionError: HTTPConnectionPool(host='www.ufcstats.com', port=80): Max retries exceeded with url: /fighter-details/1e13936d708bcff7 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000025789713F48>: Failed to establish a new connection: [WinError 10051] A socket operation was attempted to an unreachable network'))

In [15]:
fighter_history.to_csv('fight_history')

NameError: name 'fighter_history' is not defined

In [41]:
fighter_basic_stats.to_csv('fighter_basic_stats')

In [97]:
fighter_career_stats.to_csv('fighter_career_stats')

In [17]:
for url in fighter_urls:

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    base_stats = soup.findChildren('li', attrs={'class': re.compile('b-list')})

    #Retrieving careeer stats suite:
    career_stats = base_stats[5:]
    
    #Significant Strkes Landed per Minute
    SLpM = career_stats[0].text.split(':')[1].replace('%', '').strip()
    SLpM_list.append(SLpM)
    
    #Striking Accuracy %
    StrAcc =  career_stats[1].text.split(':')[1].replace('%', '').strip()
    StrAcc_list.append(StrAcc)
    
    #Striking Accuracy %
    SApM = career_stats[1].text.split(':')[1].strip()
    SApM_list.append(SApM)
    
    #Striking Defence %
    StrDef = career_stats[2].text.split(':')[1].replace('%', '').strip()
    StrDef_list.append(StrDef)
    
    #Average Takedowns Landed per 15 min
    TDAvg = career_stats[5].text.split(':')[1].strip()
    TDAvg_list.append(TDAvg)
    
    #Takedown Accuracy
    TDAcc = career_stats[6].text.split(':')[1].strip().replace('%', '')
    TDAcc_list.append(TDAcc)
    
    #Takedown Defense %
    TDDef = career_stats[7].text.split(':')[1].replace('%', '').strip()
    TDDef_list.append(TDDef)
    
    #Average Submissiongs Attempted per 15 min
    SubAvg =  career_stats[8].text.split(':')[1].strip().replace('%', '')  
    SubAvg_list.append(SubAvg)